<a href="https://colab.research.google.com/github/dharshriya15/ML-Projects/blob/main/Exercise_Indexing%2C_Selecting_%26_Assigning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041926Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0212b26102e8099c1f6b4efb3c7b713a998e3c34fc89c3e47b10b8f4ce1e591a9039e50ae4b2ace934c765f63e9b67764383d291e9fe4e9d174fdcc7c6feed13d2af68f9a2724cb65adc619bb6d45454faa5eaed9eca0e1d1ae9806844f06dc3341419923a162335009833e9ea690f824ced956e15e2f8a29abca06cbc524648eacb88a644e3717ef1d9683401431946852f1251d5718456a0261441e26a9743b01855982cd7433a0b4259fb0c83d9ce9932f3bb26a84a1f80afd81675d50ab75bd4d9132bd1648af182f40018eef157c0b0d71ac80b2fe7d8d25bdd5b719522c486ef371bfe601e0d1ad11a6cc7df838ee50a276fe895989bb986922bdb7134,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041926Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D18d3ad0d136641921b1f1e868ffe1bd048caa0661f1000c6b583e6554f09013f0a24cc4b5979d191c94bd1b24083d2cb8da31ff92d3328a43e53762834f4c5f6aa8f3ac66629413fd47f72559bdfd3baaa1a12b54abde40a2e6bd6fb118e5dd1ddda21a3c1c1bf882af73699855e4315b75024f4416413e1cbb5f30fcf9296884662396e62ca06f2dbeb039ad0009ef4fd1d485049ac1a6e8021a5ee95d21db28418e7a5590084cd3b504e5e0f30d28cf97d67e3656120f50cab58f4ef480d3034fe27049e76b3edcc0242d63d0527daf2e5461d95d514b3387286a5e96fcf07614711f75536a8f9ff4cedfef809098ea1d7a497328a73da3380ec4a3e51d0c3,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041926Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D414ebf18db8e6fee10624dc0d6b36cc4407ff2ba750484fac553fb7ce7041980ec0c007a8817131da154ed4ee607454ee434d2ed17ab169bc39f3a44db83009fbb60c8b0e74131006ccf8e71af34115b02dc22312a9b3dd54995e709ac342faf1a31c8bc4bc969a08c8be2bd8d352b974b0aeda7f9db34c58bae36ac54f063a7338f197b437b2eb905c9c85aa79bed5f0a36f773bd611a64e531eaa3b280657eb493535e8dae181fe21d0f285464b0e2f857bcc68c2ad5ef81b32bf9a9081a7919ee11553edc90fb32c8740dff7dc5e38c4a74cfc3479530c4bbb3bfd7f9c851557a5878d9de838c9bff7fbcdabd53fb1879f24877590c0cafde5050c4c0fb42,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0720e5669545d0dbcbcd10c0f037aca874a4198e87a7b2658371b4f17c2214f8ff3e40aee8621d09ac76c2a5b2d5727d89952f502d8708eeb5540e4dba6ed375cb89ff84c92401cae2959541cb257ec1f4c52747ac262bf71740e59b4423e3fa2b85ed92e1ea85c0ae983aefa17bacd70da3e6e169decf45b206419f022968b5e8938dd1708f04369dc623101675910b213635057d527918d49a9c73bdb76d5223dfd3625dad86aae657bf685040975fad449b11a4d56a5586668cdf1060d459f18ec6fbee0d9558e49e0b8505f921cbd82da4429e793db78239f012ae6be93c7e3fff868eb4f475e5557d6352e98370aee0757d8cea95abfa68fe545dfaa631,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3a1f1fd1000514e4c4d61407a3813092563812559f2b792a6dbbb1b7c4132e1a5b60c830d1ac8f59ea82d56344f2931d3e54a7e36cddf145d5b447b4c89e8835190b0ffbb4cc213a48fee805ea4cffa5f642a17962e80b96eaf986e9b8c54a922b7b829fa25871b559a56f39c3d6ff4a01cd7ac626a085f7cf95a5fd71fb1a1417b8071e2aaae85cc23c2be4315d6ce013de22dc6513e45c70e55441fff89ae74620cca532efb096ee7a2495be65a1ab7d8f7eba01e58c73c84cdeffa1e9878031e1690879be7eb44be27283a9f93bf4fdd0f07cb6cc49d4a1a24206c7664e5f40530fc28fecfd070dfedf6f0f14e9591e1f767a50a7ca4b926908c578972a97,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db03d5d0ce8dcc48959db5ca10f9e7a16189372a58a93ddd9a8072679cba1cd36d95e6fa8b1131d2c60d3591994d3fe4cad548dff0751abb2a2db7645f82c1192a866d2dce7d075b36e05a63bcbc9535e543bcadc3b04c76a906c03cafa83a3766dff695477737de84d290156b5ccd8615c753f6419eff758f622ae44e35c7b37c921c676f43479e43951f5fa66d36b9253358b2f1ac1ab2fe6ff18b0a614d84616af876c7f3b221880c811672628c9d6d74314021e242c2cea33b7469f9e2b663e9c3c99bca339e080cebc15f7a5cf74d7fc653242fb941c3e580e3f1635141f9e3ad240048ac5ee86d3dfdf7ba608b72188a3c3f774d875949524b48e8deb8e,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db6ca73d56129c1edbf9bca0d05f63c751cee5089037c4a6d8ad7e97a16b61ed2e751cc618e72da8c254bddc35d009647163d70d9139a76f755dd9b55e4a5b6dbd93f4d00389402ba2e1c4db2135b409d3d6c1a26233c32bc8f08b81b824ca4e5dbe2e82ec794c9d5f46066551b30cbae31ace0ef35fe3180d2dad391eaac13fd763162dccfdef87b48e605d049bd385c45d60557f8c310ca175452f71e8358a4564d666182f3669d738696b5460ae142afb6928b2509ecdb3c41f6e1605644b393f3d6842132f7b3aad3ad9097a9e670887a70ba180ac73163ea5fcd3ed5eaa10f9d1e0fad4e62a957aa19dbf43fbd3c72cda292a3fc555c6170580eec679eac,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D32d72e202050d97f979d7018e91d23d6a975ca619eedaa009c5de673feeced8d955d2985092e00a6d8604df0aff64342a49352200d0b780e1ed6f41bde29c6f741ece7056076d16673b2e3e650444e4f4731088464ec7d88ef5072848d46b9503163eb3b47e74fac4eb1653e3524bfcfe7dbe9f22b96bb20d62f5886579ce9615d24fe4c6a3d4eddaa3b61fd571976df3a20f831b23be438f95a0fc9d561e08156a7e7e2b37ea09f11f374504a630f62d88266753c9dd13c1dc58b73b5e0b6d0539c68bbbbd17221c2f3b9d7f8f1cbe8a011c32f95dd3df1a91c92d21e23ea1b00ff9a3b8a79ca8a49c9da82a865912f1fdf261a62be13496cb61c5d3862e5bc,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D17de274d45cd7cbdd7d6408d1a37090e9b34f262a3d873a68ab8c3d4f4ed2e034ac67c3b46771aa148ca214413bacc2134410072978ab193d9feb1933f3a27e026d94e9e2cc79c0b35ac3c5ba33dcc692b975adad259637b01407d018acc7eee2dc06085bf7af1589a96216967c182c91e8a6747795fe52094ce0125bd2cdf9c6babd51008755636d8d7a92629ba6f6395d9284c3eed7267a572dcf4b2d9f859fe732fdd142facc0b9256aad8fbcb1cb88b71d7dd48d53b890b83262185c34c5371962a99e978e6e3d934f88403ff46b40565bd314a54444bdbac2dffde4221612357f4b92dfadcb588cd3aa3a99411c0d9971a5e43a66114933ad43e49d936c,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T041927Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3bb5ed351639d5921a89f4831bdfaad424e4f0a689314ea92d68c7178198efa2feab490096547ba80b67e172a96c1b4de124e4281c9a502ad1639272fa110347cc63def16b609796972598db83d0185f86fe7128155dd0165e901a818080005d177208563e103c972b75af67da21e555600fb9966d8b24b4dd658cb4b0d9bc89aaa3f696b886b6cb47ba682f545dfc71046289259d5728144c53dd3e9a56e171e16c3effd11468455ff9fac8fcd8286038d00ba627839439e8cdc134a8d558fccb079aa21cbd7ef8e865718a48395fca3cf7e684b23c025e15bb8a4346ef808dded03129ec48d9a776a1236e2c252d77677f031bf2192237845cd328d5529b9c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/indexing-selecting-assigning).**

---


# Introduction

In this set of exercises we will work with the [Wine Reviews dataset](https://www.kaggle.com/zynicide/wine-reviews).

Run the following cell to load your data and some utility functions (including code to check your answers).

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.indexing_selecting_and_assigning import *
print("Setup complete.")

Setup complete.


Look at an overview of your data by running the following line.

In [ ]:
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Exercises

## 1.

Select the `description` column from `reviews` and assign the result to the variable `desc`.

In [ ]:
# Your code here
desc = reviews['description']

# Check your answer
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Follow-up question: what type of object is `desc`? If you're not sure, you can check by calling Python's `type` function: `type(desc)`.

In [ ]:
#q1.hint()
#q1.solution()

## 2.

Select the first value from the description column of `reviews`, assigning it to variable `first_description`.

In [ ]:
first_description = reviews['description'][0]

# Check your answer
q2.check()
first_description

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
first_description = reviews.description.iloc[0]
```
Note that while this is the preferred way to obtain the entry in the DataFrame, many other options will return a valid result, such as `reviews.description.loc[0]`, `reviews.description[0]`, and more!  


"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [ ]:
#q2.hint()
#q2.solution()

## 3.

Select the first row of data (the first record) from `reviews`, assigning it to the variable `first_row`.

In [ ]:
first_row = reviews.iloc[0]

# Check your answer
q3.check()
first_row

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

country                                                    Italy
description    Aromas include tropical fruit, broom, brimston...
                                     ...                        
variety                                              White Blend
winery                                                   Nicosia
Name: 0, Length: 13, dtype: object

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Select the first 10 values from the `description` column in `reviews`, assigning the result to variable `first_descriptions`.

Hint: format your output as a pandas Series.

In [ ]:
first_descriptions = reviews.description.iloc[:10]

# Check your answer
q4.check()
first_descriptions

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
first_descriptions = reviews.description.iloc[:10]
```
Note that many other options will return a valid result, such as `desc.head(10)` and `reviews.loc[:9, "description"]`.    


0    Aromas include tropical fruit, broom, brimston...
1    This is ripe and fruity, a wine that is smooth...
                           ...                        
8    Savory dried thyme notes accent sunnier flavor...
9    This has great depth of flavor with its fresh ...
Name: description, Length: 10, dtype: object

In [ ]:
#q4.hint()
#q4.solution()

## 5.

Select the records with index labels `1`, `2`, `3`, `5`, and `8`, assigning the result to the variable `sample_reviews`.

In other words, generate the following DataFrame:

![](https://storage.googleapis.com/kaggle-media/learn/images/sHZvI1O.png)

In [ ]:
sample_reviews = reviews.loc[reviews.index.isin([1,2,3,5,8])]

# Check your answer
q5.check()
sample_reviews

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel


In [ ]:
#q5.hint()
#q5.solution()

## 6.

Create a variable `df` containing the `country`, `province`, `region_1`, and `region_2` columns of the records with the index labels `0`, `1`, `10`, and `100`. In other words, generate the following DataFrame:

![](https://storage.googleapis.com/kaggle-media/learn/images/FUCGiKP.png)

In [ ]:
df = reviews.loc[reviews.index.isin([0,1,10,100]),reviews.columns.isin(['country','province','region_1','region_2'])]

# Check your answer
q6.check()
df

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,province,region_1,region_2
0,Italy,Sicily & Sardinia,Etna,NaN
1,Portugal,Douro,NaN,NaN
10,US,California,Napa Valley,Napa
100,US,New York,Finger Lakes,Finger Lakes


In [ ]:
#q6.hint()
#q6.solution()

## 7.

Create a variable `df` containing the `country` and `variety` columns of the first 100 records.

Hint: you may use `loc` or `iloc`. When working on the answer this question and the several of the ones that follow, keep the following "gotcha" described in the tutorial:

> `iloc` uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded.
`loc`, meanwhile, indexes inclusively.

> This is particularly confusing when the DataFrame index is a simple numerical list, e.g. `0,...,1000`. In this case `df.iloc[0:1000]` will return 1000 entries, while `df.loc[0:1000]` return 1001 of them! To get 1000 elements using `loc`, you will need to go one lower and ask for `df.iloc[0:999]`.

In [ ]:
df = reviews.iloc[0:100,reviews.columns.isin(['country','variety'])]

# Check your answer
q7.check()
df

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
cols = ['country', 'variety']
df = reviews.loc[:99, cols]
```
or 
```python
cols_idx = [0, 11]
df = reviews.iloc[:100, cols_idx]
```


,country,variety
0,Italy,White Blend
1,Portugal,Portuguese Red
...,...,...
98,Italy,Sangiovese
99,US,Bordeaux-style Red Blend


In [ ]:
#q7.hint()
#q7.solution()

## 8.

Create a DataFrame `italian_wines` containing reviews of wines made in `Italy`. Hint: `reviews.country` equals what?

In [ ]:
italian_wines = reviews.loc[reviews.country.isin(['Italy'])]

# Check your answer
q8.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q8.hint()
#q8.solution()

## 9.

Create a DataFrame `top_oceania_wines` containing all reviews with at least 95 points (out of 100) for wines from Australia or New Zealand.

In [ ]:
top_oceania_wines = reviews.loc[(reviews['points'] >= 95) & (reviews.country.isin(['Australia','New Zealand']))]

# Check your answer
q9.check()
top_oceania_wines

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
345,Australia,This wine contains some material over 100 year...,Rare,100,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards
346,Australia,"This deep brown wine smells like a damp, mossy...",Rare,98,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscadelle...,Muscadelle,Chambers Rosewood Vineyards
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122507,New Zealand,"This blend of Cabernet Sauvignon (62.5%), Merl...",SQM Gimblett Gravels Cabernets/Merlot,95,79.0,Hawke's Bay,NaN,NaN,Joe Czerwinski,@JoeCz,Squawking Magpie 2014 SQM Gimblett Gravels Cab...,Bordeaux-style Red Blend,Squawking Magpie
122939,Australia,Full-bodied and plush yet vibrant and imbued w...,The Factor,98,125.0,South Australia,Barossa Valley,NaN,Joe Czerwinski,@JoeCz,Torbreck 2013 The Factor Shiraz (Barossa Valley),Shiraz,Torbreck


In [ ]:
#q9.hint()
#q9.solution()

# Keep going

Move on to learn about **[summary functions and maps](https://www.kaggle.com/residentmario/summary-functions-and-maps)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*